In [7]:
import numpy as np
import pandas as pd
import os.path
import subprocess
import math
import datetime
import matplotlib
from copy import copy, deepcopy
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
from datetime import datetime, timedelta

In [8]:
#settings
folderpath='/Users/B.Suryanarayanan/Documents/Git/Lake_Ice'
start_date=datetime(2017,12,6)
start_date_scen=datetime(2018,1,9)
num_days=start_date_scen-start_date
num_days=num_days.days
num_days_scen=14

In [9]:

# ---- scenario - INPUT / ADJUSTMENT / CALIBRATION
scenprec_corrfact=1.0              # Factor to correct precipitation in second week of scenario - normally strongly overestimated...

ice_thickness_scen=25              # (cm) Ice thickness (measured) at start of scenario period  (default: -9999)
#ice_thickness_scen=-np.inf

snow_thickness_scen=3             # (cm w.==!) snow thickness (measured) at start of scenario period  (default: -9999)
#snow_thickness_scen=-np.inf

snow_dens_scen=400                # (kg m-3) snow density (measured/estimated)  at start of scenario period  (default: -9999)
#snow_dens_scen=-np.inf

slushice_thickness_scen=0               # (cm) thickness of refrozen ice ("Sandwich") at start of scenario period  (default: -9999)
#slushice_thickness_scen=-np.inf

# ------------------

twater_ini=[4,4,20]       # [Tmin,Tmax,break(m)]

int_slush=[datetime(2018,1,3,12),datetime(2018,1,4,12),datetime(2018,1,9,12),-np.inf]       # dates to insert slush

vol_slush=[0.003,0.017,0.01,-np.inf]            # volumes of inserted slush in m w.e. (same length as int_slush)

dtdz=-0.006       # temperature gradient (deg C km-1)
hstat=1709         # elevation of weather station

alpha_ice=0.3         # albedo of ice
alpha_water=0.12      # albedo of water
alpha_snow0=0.92      # albedo of fresh snow
alpha_snow1=0.5       # albedo of very old snow

turbulent_fact=3.5  # 5 # factor to increase transfer of water heat to base of ice (to be calibrated)

corrprec=1      # Korrektur des Niederschlags

lakedepth=25     # depth of lake m

first_freezeparam=1.0        # [-] Parameter to accelerate or decelerate the first freezing of the lake

# temperature of river entering the lake and lake geometry
topo=[0.78,1768,44,0.0172]        # area, elevation, max depth, volume
flow=1     #5.8                    # [m3/s] river charactersitic inflow
flow_tcorr=-5    # [deg C] correct temperature of stream flow for elevation / snow melt relative to air temperature
flow_tmin=0      # [deg C] minimum temperature of stream

wwin=[1.2,1,0.8]     # weighting of lake water layering in different depths

elev_lake=topo[1]     # elevation of lake


# number of iterations  for heat conduction (per hour)
iterations_si=600     # snow / ice
iterations_w=50     # water

# number of iterations for snow surface temperature
dt_snoit=120    # s

freq_plots=6     # (hours) frequency of output plots


# -----------------------------
# parameters / constants

kice=2.33       # conductivity ice [J s-1 K-1 m-1]
kwater=0.56     # conductivity water
kair=0.001

cice=1890000     # heat capacity of ice
cair=1297
cwater=4217700    # [J m-3 K-1]

emissivity_atm=0.95
emissivity_ice=0.966
emissivity_water=0.957
emissivity_snow=0.97

attenuation_water=1       # m-1
attenuation_ice0=1.0         # m-1
attenuation_snow=10     # m-1   ??? 10
at_rho=250
fact_at=0.1                # increase in ice attenuation per day (ageing)

rho=917   # ice density
rho_water=1000
rho_snow0=70    # density of fresh snow
rho_max=400

compression='y'
rho_compression=450    # kg m-3


L=334000*rho    # [J m-3] latent heat of fusion
Lhe=2260000    # [J kg-1] latent heat of evaporation
Lhs=2594000    # [J kg-1] latent heat of sublimation

bulk_exchange_s=0.3*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=0.8*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)

bulk_exchange_s=1.5*pow(10,-3)      # bulk exchange coefficients for sensible heat flux (to be calibrated)
bulk_exchange_l=1.5*pow(10,-3)      # bulk exchange coefficients for latent heat flux (to be calibrated)


# -----------------------
# settings

dz=0.005      # nodespacing (ice)
dzs=0.0025    # snow
dzw=0.2      # water

nodes=[lakedepth/dzw,300,300]     # water, ice, snow

dt=3600     # [s] timestep

s_b=5.67*pow(10,-8)    # stefan boltzmann

t_wmin=4     # minimum temperature of water (density)

meltlast=0

In [10]:
os.chdir(folderpath)
df=pd.read_csv('./data/interim/scenario.csv')
df['When']=df['Date'] + ' ' + df['Time']
df['When'] = pd.to_datetime(df['When'],format='%Y-%m-%d %H:%M:%S')
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
df.tail()

,Unnamed: 0,Rad,Tair,RH,Wind,Pressure,Prec,Cl,Date,Time,huma,rho_air,When
1147,1147,0.0,-7.4,78.8,1.3,82610.0,0.000156,0.40335,2018-01-22,2018-05-24 19:00:00,0.00158,1.09758,2018-01-22 19:00:00
1148,1148,0.0,-7.4,75.1,1.4,82660.0,0.000156,0.40335,2018-01-22,2018-05-24 20:00:00,0.00168,1.09241,2018-01-22 20:00:00
1149,1149,0.0,-9.8,75.6,0.6,82630.0,0.000156,0.40335,2018-01-22,2018-05-24 21:00:00,0.00178,1.08954,2018-01-22 21:00:00
1150,1150,0.0,-9.8,80.2,0.8,82590.0,0.000156,0.40335,2018-01-22,2018-05-24 22:00:00,0.00195,1.08736,2018-01-22 22:00:00
1151,1151,0.0,-9.8,94.7,0.7,82560.0,0.000156,0.40335,2018-01-22,2018-05-24 23:00:00,0.00230,1.08697,2018-01-22 23:00:00


In [11]:
#variables
ice=np.full(nodes[1],-np.inf)
snow=np.full((3,nodes[2]),-np.inf)
nodes=list(map(int, nodes))
fr=0
sn=0
im=0
u=0
y=0
c=0
i_slush=0
Qbmelt=0
Qswi_i=0.0
ci_icest=0
water=np.zeros(shape=nodes[0])
water+=twater_ini[0]

for i in range(nodes[0]-1,-1,-1):
    d=i*dzw
    if d< twater_ini[2]:
        water[i]=water[i]+((((twater_ini[2]/dzw)-i)*(twater_ini[1]-twater_ini[0]))/(twater_ini[2]/dzw))

slush=np.full((3,nodes[2]),0.0)
icethick=np.full(df.shape[0],0.0)
load=np.full((2,df.shape[0]),0.0)
mtemp=np.full((3,df.shape[0]),0.0)
stype=np.full(df.shape[0],0.0)
snowthick=deepcopy(stype)
icet=deepcopy(stype)
snow_albedo=deepcopy(stype)
slush_thick=deepcopy(icethick)
i_age=np.full(df.shape[0],-np.inf)

dfo=pd.DataFrame({'Type' : []})
dfo['Temp']=-np.inf
dfo['Date']=df['When'].dt.date
dfo['Time']=df['When'].dt.time
dfo['Icethick']=0
dfo['Snowthick']=0

In [ ]:
with PdfPages('./data/processed/figures.pdf') as pdf:
    for index in range(0,df.shape[0]):
        ii=np.where(snow[0,]!=-np.inf)
        ci=ii[0].size
        jj=np.where(ice!=-np.inf)
        cj=jj[0].size
        hh=np.where(water!=-np.inf)
        ch=hh[0].size
        if ci>0:
            tt=snow[0,ci-1]
            print(df.loc[index,'When'],'Snow',ci)
            print(snow[0,0:ci])
            dfo.loc[index,'Type']='Snow'

        if ci==0 and cj>1:
            tt=ice[0]
            print(df.loc[index,'When'],'Ice',cj)
            print(ice[0:5])
            dfo.loc[index,'Type']='Ice'
        if cj==0:
            tt=water[0]
            print(df.loc[index,'When'],'Water')
            print(water[0:5])
            dfo.loc[index,'Type']='Water'
        dfo.loc[index,'Temp']=tt

        if np.isnan(tt) or tt==-np.inf:
            print(snow[0,],cj)
            exit()

        ew=6.1094*np.exp((17.625 * tt)/(tt+243.04))*100
        hum0=0.622*ew/abs(df.loc[index,'Pressure']-ew)
    # *************************************************
    # *************************************************
    # SNOW
        tes=deepcopy(snow[0,])
        tls=deepcopy(tes)

        # accumulation
        t_threshold=1.25
        ptt=df.loc[index,'Prec']
        if df.loc[index,'Tair']>t_threshold+1:
            df.loc[index,'Prec']=0
        if df.loc[index,'Tair']<=t_threshold+1 and df.loc[index,'Tair']>t_threshold-1:
            df.loc[index,'Prec']=df.loc[index,'Prec']*(1-(df.loc[index,'Tair']-(t_threshold-1))/2)

        df.loc[index,'p_liq']=ptt-df.loc[index,'Prec'] #Liquid Prec

        if ice[0]==-np.inf:
            df.loc[index,'Prec']=0
        
        sn=sn+round(df.loc[index,'Prec'],4) #Solid Prec
        print("Snow",sn)

        if sn>=dzs:
            if snow[0,0]==-np.inf:
                snow[0,0]=ice[0]
            sn=sn-dzs
            l=np.where(snow[0,]==-np.inf)
            l=l[0]
            snow[0,l[0]]=min([df.loc[index,'Tair'],0])
            snow[1,l[0]]=rho_snow0/1000

            #artificial compression of snow
            if compression=='y':
                snow[1,l[0]]=rho_compression/1000

            snow[2,l[0]]=1/24

        #update snow density and albedo
        ts_snow=21.9
        l=np.where(snow[0,]!=-np.inf)
        l=l[0]
        if l.size!=0:
            for i in range(0,l.size):
                if snow[1,i]<rho_max/1000:
                    snow[1,i]=(rho_max-(rho_max-rho_snow0)*math.exp(-0.03*pow((0.01*rho_max),0.5)*snow[2,i]))/1000
                    snow[1,0]=0.2

                if snow[2,i]!=-np.inf:
                    alpha_snow=alpha_snow1+(alpha_snow0-alpha_snow1)*math.exp((-1)*snow[2,i]/ts_snow)
                else:
                    alpha_snow=0.85
                if snow[2,i]!=-np.inf:
                    snow[2,i]=snow[2,i]+1/24

        else:
            alpha_snow=0.85

        # arrays for capacity and conductivity depending on density for snow
        csnow=np.full(nodes[2],0.0)
        ksnow=deepcopy(csnow)
        for i in range(0,nodes[2]):
            if snow[0,i] != -np.inf:
                csnow[i]=(1-snow[1,i])*cair+snow[1,i]*cice
            if snow[0,i] != -np.inf:
                ksnow[i]=(2.93*pow(snow[1,i],2))+0.01   # according to Mellor (1977)

        # update capacity and conductivity if slush is present
            if slush[0,i]!= 0:
                c=slush[1,i]*cwater+(1-slush[1,i])*cice
                k=slush[1,i]*kwater+(1-slush[1,i])*kice                
                ff=(slush[0,i]/slush[2,i])*(1-snow[1,i])
                csnow[i]=(1-snow[1,i]-ff)*cair+snow[1,i]*cice+ff*c
                ksnow[i]=(1-snow[1,i]-ff)*kair+snow[1,i]*kice+ff*k

        # -------  energy balance snow
        Qswi=(1-alpha_snow)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_snow*df.loc[index,'Rad']   # outgoing short wave

        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*pow((df.loc[index,'Tair']+273.15),4)
        Qe=df.loc[index,'rho_air']*Lhs*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']
        if df.loc[index,'Tair']> 1 :
            Qp=df.loc[index,'p_liq']*(df.loc[index,'Tair']-1)*cwater/dt
        else:
            Qp=0
        gamma=math.exp(-attenuation_snow*dzs/(at_rho/1000.))
        # iterate for snow surface temperature
        if snow[0,0]== -np.inf :
            dt_snoit_tt=1
        else:
            dt_snoit_tt=dt_snoit

        snit=dt/dt_snoit_tt
        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        snit=int(snit)
        for it in range(0,snit):
            if snow[0,0]==-np.inf :
                tsno=0
            else:
                if it==0:
                    tsno=snow[0,l[ci-1]]

                else:
                    tsno=tt_ts
                tt_ts=tsno

            Qlwo=(-1)*emissivity_snow*s_b*pow((tsno+273.15),4)
            Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-tsno)*df.loc[index,'Wind']
            Q0snow=(1-gamma)*Qswi+Qlwi+Qlwo+Qc+Qe+Qp

        # calculate temperature of uppermost layer with energy balance
            if ci > 1 :
                tt_ts=tt_ts+Q0snow*dt_snoit_tt/(csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]]))

        # done iteration for snow surface temperature

        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 1 :
            # attribute surface temperature
            tls[l[ci-1]]=tt_ts
            tls[0]=ice[0]

            for i in range(ci-2,0,-1):
                tls[l[ci-2]]=tls[l[ci-1]]+(math.exp(-attenuation_snow*(i+1)*dzs/(at_rho/1000))-math.exp(-attenuation_snow*i*dzs/(at_rho/1000)))*Qswi*dt/(csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]]))

            Qswi_s=Qswi*math.exp(-attenuation_snow*(ci-1)*dzs/(at_rho/1000))

            # ----- snow evaporation
            if Qe < 0 :
                sn=sn+Qe*dt/(Lhs*1000)

            # ----- snow melt
            if Q0snow > 0 and 0<=tls[l[ci-1]] :
                Qmelt=Q0snow+(tls[l[ci-1]]*csnow[l[ci-1]]*(dzs/snow[1,l[ci-1]])/dt)
                melt=Qmelt/L*dt
                if slush[1,l[ci-1]]!=0:
                    m=melt*(dzs/(dzs+slush[0,l[ci-1]]*(slush[1,l[ci-1]])))
                else:
                    m=melt
                sn=sn-m
                tls[l[ci-1]]=0
            else:
                melt=meltlast
            # reduce snow
            if sn <= -dzs :
                sn=sn+dzs
                tls[l[ci-1]]=-np.inf
            hh=np.where(slush[1,] == 0)
            hh=hh[0]
            ch=hh.size
            print("Slush Counter5",ch)
            print("Slush",np.sum(slush[0,hh])*1.25)
            
            l=np.where(tls != -np.inf)
            l=l[0]
            ci=l.size
            
            # add liquid precipitation to melt water
            melt=melt+round(df.loc[index,'p_liq'],4)
            
            if df.loc[index,'p_liq']>0:
                print("Added prec", melt)
            
            # ----- refreezing of melt water
            if melt > 0:
                for j in range(ci-2,0,-1) :
                    re=(-1)*snow[0,j]*csnow[j]*(dzs/snow[1,j])/L
                    if melt > re :
                        tls[j]=0
                    else:
                        if re > 0 :
                            tls[j]=(1-melt/re)*snow[0,j]
                    if melt > re :
                        melt=melt-re
                    else:
                        if re > 0:
                            melt=0

            # ----- slush formation (melt water in snow pore space)
            if melt > 0 :
                print("Slush formation")
                for j in range(1,ci):
                    print(j,"slush0",slush[0,j])
                    sp=dzs/snow[1,j]*(1-snow[1,j])
                    a=sp-slush[0,j]
                    slush[2,j]=sp
                    if melt < a and melt != 0 :
                        slush[0,j]=slush[0,j]+melt
                        melt=0
                        slush[1,j]=1
                        print('Refrozen slush at', j,slush[0,j])
                    if melt > a :
                        melt=melt-a
                        slush[0,j]=sp
                        print('Slush at', j, slush[0,j],slush[1,j])
                    if melt < 0 :
                        melt=0
                    if slush[0,j] < 0 :
                        slush[0,j]=0
            hh=np.where(slush[1,] == 0)
            hh=hh[0]
            ch=hh.size
            print("Slush Counter4",ch)
            print("Slush",np.sum(slush[0,hh])*1.25)
            jj=np.where(snow[0,] == -np.inf)
            jj=jj[0]
            slush[0,jj]=0
            if jj.size>0:
                if jj[0]<= 1 :
                    slush[0,0:2]=dzs
            
            # ----- manual slush injection (melt water in snow pore space) - injection at 12.00
            if vol_slush[y] != -np.inf :
                if df.loc[index,'When'] in int_slush:
                    #print("Vol_in", slush[0,j],slush[1,j],slush[2,j])
                    ij=vol_slush[y]
                    y=y+1
                    print("Vol",ij)
                    for j in range(1,ci) :
                        #print("Snow1,slush0",snow[1,j],slush[0,j])
                        sp=dzs/snow[1,j]*(1-snow[1,j])
                        a=sp-slush[0,j]
                        if ij > 0 :
                            if ij < a :
                                slush[0,j]=slush[0,j]+ij
                                ij=0
                                slush[1,j]=1
                                slush[2,j]=sp
                            else:
                                slush[0,j]=slush[0,j]+a
                                ij=ij-a
                                slush[1,j]=1
                                slush[2,j]=sp
                        #print("Vol_in2", j, slush[0,j],slush[1,j],slush[2,j])

            # ----- refreezing of slush
            for j in range(1,ci) :
                if slush[0,j] != 0 and slush[1,j] > 0 and tls[j] < 0 :
                    re=(-1)*tls[j]*csnow[j]*(dzs/snow[1,j])/L
                    if (slush[0,j]*slush[1,j]) > re :
                        tls[j]=0
                        slush[1,j]=slush[1,j]-(re/(slush[0,j]*slush[1,j]))*slush[1,j]
                        print("Refrozen slush at",j,slush[0,j])
                    else:
                        if re > 0 :
                            tls[j]=(1+(slush[0,j]*slush[1,j])/re)*tls[j]
                        slush[1,j]=0
                        print('More slush at',j,slush[0,j])

            hh=np.where(slush[1,] == 0)
            hh=hh[0]
            ch=hh.size
            print("Slush Counter3",ch)
            print("Slush",np.sum(slush[0,hh])*1.25)
            
            l=np.where(tls != -np.inf)
            l=l[0]
            ci=l.size
            tes=deepcopy(tls)

            # -----------------------
            # heat conduction
            
            tll=deepcopy(tls)
            tel=deepcopy(tes)
            for it in range(0,iterations_si) :
                for j in range(0,ci-1) :
                    if j == 0 :
                        tel[j]=tll[j]-(dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j]-tll[j+1]))/(pow((dzs/snow[1,j]),2))/2
                    else:
                        tel[j]=tll[j]+((dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j-1]-tll[j])/pow(dzs/snow[1,j],2))-(dt/iterations_si*ksnow[j]/(csnow[j])*(tll[j]-tll[j+1])/pow(dzs/snow[1,j],2)))/2
                tll=deepcopy(tel)

            tes=deepcopy(tel)
            ice[0]=tes[0]   # setting ice surface temperature to value of lowermost snow layer!
            snow[0,]=deepcopy(tes)
            snow[0,0]=tls[0]

            if l.size > 0 :
                snow[0,l[ci-1]]=tls[l[ci-1]]
            meltlast=melt

        # correct and reset
        if snow[0,1] == -np.inf :
            snow.T[0,]=-np.inf
        l=np.where(np.logical_and(snow[0,]!=-np.inf,snow[0,]<-40))
        l=l[0]

        if l.size > 0 :
            snow[0,l]=-40

        l=np.where(np.logical_and(snow[0,]!=-np.inf,snow[0,]>0))
        l=l[0]
        if l.size > 0 :
            snow[0,l]=0

        # ----  forcing layers to measured thicknesses at begin of scenario period
        if df.loc[index,'When']==start_date_scen:
            if snow_thickness_scen != -np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    snow_thickness_scen_ind=int(snow_thickness_scen/dzs/100)
                    if l.size < snow_thickness_scen_ind:
                        for h in range(l.size,snow_thickness_scen_ind):
                            snow.T[h,]=snow.T[l[l.size-1],]
                    else:
                        for h in range(snow_thickness_scen_ind, l.size):
                            snow.T[h,]=-np.inf
            if snow_dens_scen !=-np.inf and snow_thickness_scen !=-np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    snow_thickness_scen_ind=int(snow_thickness_scen/dzs/100)
                    for h in range(0,snow_thickness_scen_ind):
                        snow[1,h]=snow_dens_scen/1000

            if slushice_thickness_scen != -np.inf:
                l=np.where(snow[0,]!=-np.inf)
                l=l[0]
                if l.size > 0 :
                    slushice_thickness_scen_ind=int(slushice_thickness_scen/dzs/100)
                    if l.size<slushice_thickness_scen_ind:
                        for h in range(l.size,slushice_thickness_scen_ind):
                            slush.T[h,]=slush.T[l[l.size-1],]
                    else:
                        for h in range(slushice_thickness_scen_ind,l.size):
                            slush.T[h,]=0
        
        # *************************************************
        # ICE
        # age of ice sheet
        ii=np.where(ice != -np.inf)
        ii=ii[0]
        ci=ii.size
        jj=np.where(ice == -np.inf)
        jj=jj[0]
        i_age[jj]=-np.inf
        for i in range(0,ci):
            if i_age[i] == -np.inf:
                i_age[i]=1/24
            else:
                i_age[i]=i_age[i]+1/24

        if ci > 0 :
            attenuation_ice=attenuation_ice0+np.mean(i_age[ii])*fact_at
        else:
            attenuation_ice=attenuation_ice0

        # energy balance ice
        Qswi=(1-alpha_ice)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_ice*df.loc[index,'Rad']
        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*pow((df.loc[index,'Tair']+273.15),4)
        Qe=df.loc[index,'rho_air']*Lhs*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']                                     #huma==hum0???
        Qp=0
        gamma=math.exp(-attenuation_ice*dz)
        
        # iterate for ice surface temperature
        if snow[0,1] == -np.inf :
            dt_iceit=dt_snoit
        else:
            dt_iceit=dt
        snit=(dt/dt_iceit)

        for it in range(0,int(snit)) :
            if ice[0] == -np.inf :
                tice=0
            else:
                tice=ice[0]
            if it > 1 :
                tice=tt_ts
            else:
                tt_ts=tice

            Qlwo=(-1)*emissivity_ice*s_b*pow((tice+273.15),4)     # long-wave outgoing
            Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-tice)*df.loc[index,'Wind']   # sensible heat flux
            Q0ice=(1-gamma)*(Qswi)+Qlwi+Qlwo+Qc+Qe+Qp          # energy balance of top layer
            tt_ts=tt_ts+Q0ice*dt_iceit/(cice*dz)

        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 0 :
            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
        
            te=deepcopy(ice)
            tl=deepcopy(te)
            # calculate temperature of uppermost layer with energy balance
            if snow[0,1] == -np.inf :
                tl[0]=tt_ts# apply surface temperature from iteration

                # adapting ice temperature based on direct radiation absorbed within ice sheet
                
#                 for i in range(1,ci-1):
#                     tl[i]=tl[i]+(math.exp(-attenuation_ice*(i-1)*dz)-math.exp(-attenuation_ice*i*dz))*Qswi*dt/(cice*dz)
#                 Qswi_i=Qswi*math.exp(-attenuation_ice*(ci-2)*dz)    # radiation below ice sheet
                
            # below snow coverage - temperature change calculated with heat fluxes from above and below
            if snow[0,1] != -np.inf :
                for i in range(0,ci-1):
                    tl[i]=tl[i]+(math.exp(-attenuation_ice*(i-1)*dz)-math.exp(-attenuation_ice*i*dz))*Qswi_s*dt/(cice*dz)
                Qswi_i=Qswi_s*math.exp(-attenuation_ice*(ci-2)*dz)   # radiation below ice sheet

                snow[0,0]=tl[0]     # equalizing ice surface and bottom snow temperature
            
            # ----- ice evaporation
            if Qe < 0 and snow[0,1] == -np.inf :
                if 0<= i_slush :
                    im=im+Qe*dt/(Lhs*917)
                else:
                    i_slush=i_slush+Qe*dt/(Lhw*1000)
                if i_slush < 0 :
                    i_slush=0

            # ----- ice melt
            if Q0ice > 0 and tl[0] >= 0 :
                Qmelt=Q0ice+(tl[0]*cice*dz/dt)
                if Qmelt > 0 :
                    imelt=Qmelt/L*dt
                else:
                    imelt=0
                im=im-imelt
                tl[0]=0
                i_slush=i_slush+imelt
            else:
                melt=0

            # reduce ice cover
            if im <= -dz :
                im=im+dz
                for i in range(0,ci-1):
                    ice[i]=ice[i+1]
                ice[ci-1]=-np.inf
                ii=np.where(ice != -np.inf)
                ii=ii[0]
                ci=ii.size
                tl=deepcopy(ice)
            # ---- freezing of slush at ice surface
            print("freezing of slush at ice surface", slush[0,1],slush[1,1])
            print("Temp",tl[0])
            if slush[0,1] > 0 and tl[0] < 0 :
                re=(-1)*tl[0]*cice*dz/L

                if slush[0,1]*slush[1,1] > re :
                    tl[0]=0
                    slush[1,1]=slush[1,1]*(re/(slush[0,1]*slush[1,1]))
                    print("Slush Refrozen at 1",slush[0,1],slush[1,1])
                else:
                    if re > 0 :
                        tl[0]=tl[0]+tl[0]*((slush[0,1]*slush[1,1])/re)
                    slush[1,1]=0
                    print("More slush at 1", slush[0,1])
            hh=np.where(slush[1,] == 0)
            hh=hh[0]
            ch=hh.size
            print("Slush Counter2",ch)
            print("Slush",np.sum(slush[0,hh])*1.25)
            
            # ---- freezing liquid water on bare ice surface or within ice sheet!
            if i_slush > 0 :
                re=0
                for j in range(0,ci) :
                    if tl[j] < 0 :
                        re=(-1)*tl[j]*cice*dz/L
                    if i_slush > re :
                        tl[j]=0
                        i_slush=i_slush-re
                    else:
                        if re > 0 :
                            tl[j]=tl[j]+tl[j]*(i_slush/re)
                        i_slush=0
            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
            # ------- heat conduction
            print("JJ",te[0:ci])
            tll=deepcopy(tl)
            tel=deepcopy(te)
            for it in range(0,iterations_si) :
                for j in range(1,ci-1) :
                    tel[j]=tll[j]+((dt/iterations_si*kice/(cice)*(tll[j-1]-tll[j])*pow(dz,-2))-(dt/iterations_si*kice/(cice)*(tll[j]-tll[j+1])*pow(dz,-2)))/2
                tll=deepcopy(tel)
                
            te=deepcopy(tel)
            print(te[0:ci])
            jj=np.where(tl == -np.inf)
            jj=jj[0]
            cj=jj.size
            if cj > 0 :
                te[jj]=-np.inf
            # --- check for positive ice temperatures
            jj=np.where(te>0)
            jj=jj[0]
            cj=jj.size
            
            hh=np.where(np.logical_and(te<0,te!=-np.inf))
            hh=hh[0]
            ch=hh.size
            if cj > 0 and ch == 0 :
                fr=fr-np.sum(te[jj])*cice*dz/L
                te[jj]=0
            
            if cj > 0 and ch > 0 :
                p=np.sum(te[jj])/ch
                kk=np.where(np.logical_and(te < -p, te != -np.inf) )
                kk=kk[0]
                ck=kk.size
                if ck > 0 :
                    te[kk]=te[kk]+np.sum(te[jj])/ck
                else:
                    fr=fr-np.sum(te[jj])*cice*dz/L
                    te[jj]=0
            # ---  freezing / thawing at base!
            if ci > 1 :
                ice_flux=(-1)*min(te[ii])*cice*dz/dt
            else:
                ice_flux=0.
            if ice_flux < 0 :
                ice_flux=0
            if ci > 1 :
                fr=fr+(ice_flux+(turbulent_fact*kwater*(water[0]-water[1])/(dzw))-Qbmelt)*dt/L
            else:
                tl[0]=-np.inf
            if fr > dz :
                fr=fr-dz
                te[ci]=0
            if fr < (-1)*dz :
                fr=fr+dz
                te[ci-1]=-np.inf
                print("Melted")
                if ci > 1 :
                    te[ci-2]=0
            ice=deepcopy(te)
            ice[0]=tl[0]   # ice loop
        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
            

        if ice[0] == -np.inf :
            ice[1]=-np.inf
        if snow[0,0] == -np.inf :    #Correction
            snow[0,1]=-np.inf
        
        
        # Break up of ice cover if it is completely temperate and reasonably thin
        l=np.where(ice != -np.inf)
        l=l[0]
        ci=l.size
        if ci > 0 and ci < 4 :

            if min(ice[l])==0 :
                ice[l]=-np.inf
                print("Broken")
                
        
        
        
        #correct and reset
        ii=np.where(np.logical_and(ice != -np.inf, ice<-30))
        ii=ii[0]
        ci=ii.size
        if ci>0:
            ice[ii]=-30
        
        #forcing ice thickness at beginning of scenario period
        if df.loc[index,'When']==start_date_scen:
            if ice_thickness_scen != -np.inf:
                ice_thickness_scen_ind=int(ice_thickness_scen/dz/100)
                hh=np.where(np.logical_and(ice != -np.inf, ice<-30))
                hh=hh[0]
                ch=hh.size
                if ch < ice_thickness_scen_ind:
                    for h in range(ch,ice_thickness_scen_ind):
                        ice[h]=0
                else:
                    for h in range(ice_thickness_scen_ind,ch):
                        ice[h]=-np.inf

        # *************************************************
        # *************************************************
        # WATER

        # iterate for snow surface temperature
        ts_water=water[0]
        # energy balance water
        Qswi=(1-alpha_water)*df.loc[index,'Rad']
        Qswo=(-1)*alpha_water*df.loc[index,'Rad']   
        Qlwo=(-1)*emissivity_water*s_b*pow((ts_water+273.15),4)
        Qlwi=(0.68+0.0036*pow(df.loc[index,'huma'],0.5))*(1+0.18*pow(df.loc[index,'Cl'],2))*emissivity_atm*s_b*(pow(df.loc[index,'Tair']+273.15,4))
        Qc=df.loc[index,'rho_air']*cair*bulk_exchange_s*(df.loc[index,'Tair']-ts_water)*df.loc[index,'Wind']
        Qe=df.loc[index,'rho_air']*Lhe*bulk_exchange_l*(df.loc[index,'huma']-hum0)*df.loc[index,'Wind']
        Qp=0
        gamma=math.exp(-attenuation_water*dz)
        Q0water=Qswi+Qlwi+Qlwo+Qc+Qe+Qp
        l=np.where(water != -np.inf)
        l=l[0]
        ci=l.size
        tew=deepcopy(water)
        tlw=deepcopy(tew)
        
        # mixing of water during summer - 	include layering of water!
        if ice[0] == -np.inf :
            ww=np.full(ci,1.0)
            ww[0:11]=wwin[0]
            ww[10:21]=wwin[1]
            ww[ci-10:ci]=wwin[2]
            ww=ww/np.mean(ww)
            cj=ci
            if water[0] < t_wmin :
                ww=np.full(ci,0.0)
                ww[0]=1
                cj=1
            if Q0water < 0 :
                jj=np.where(water > t_wmin)
                jj=jj[0]
                cj=jj.size
                ww=np.full(ci,0.0)
                if cj > 0 :
                    ww[jj]=1
                else:
                    ww[0]=1
                if cj == 0 :
                    cj=1

            for j in range(0,cj):
                tlw[j]=tlw[j]+Q0water*dt/(cwater*cj*dzw)*ww[j]

            # !!! heat input by rivers and rain !!!
            tr=(df.loc[index,'Tair']+flow_tcorr)
            if tr < flow_tmin :
                tr=flow_tmin
            di_temp=tr-np.mean(water)
            hrain=(df.loc[index,'Prec']*corrprec)*1000000.*topo[0]*di_temp  # cwater gekuerzt!
            tlw=tlw+hrain/(topo[3]*pow(10,9))
            ran_riv=168
            tt=min([ran_riv,index])
            triv=np.mean(df.loc[index-tt:index+1,'Tair'])+flow_tcorr
            if triv < flow_tmin :
                triv=flow_tmin
                di_temp=triv-np.mean(water)
            hriv=flow*dt*di_temp  # cwater gekuerzt!
            tlw=tlw+hriv/(topo[3]*pow(10,9))
        
        #If Ice Covered
        if ice[0] != -np.inf :
            tlw[0]=0
            Qbmelt=Qswi_i*(1-gamma)
            for i in range(1,ci):
                tlw[i]=tlw[i]+(math.exp(-attenuation_water*(i-1)*dz)-math.exp(-attenuation_water*i*dz))*Qswi_i*dt/(cwater*dzw)

        tll=deepcopy(tlw)
        tel=deepcopy(tew)
        
        for it in range(0,iterations_w):
            for j in range(1,ci-1) :
                tel[j]=tll[j]+((dt/iterations_w*kwater/(cwater)*(tll[j-1]-tll[j])/pow(dzw,2))-(dt/iterations_w*kwater/(cwater)*(tll[j]-tll[j+1])/pow(dzw,2)))/2
            tll=deepcopy(tel)
        tew=deepcopy(tel)
        tew[0]=tlw[0]

        # try to eliminate values < t_wmin
        for j in range(0,ci) :
            if tew[j] < t_wmin and max(tew) > t_wmin :
                tp=t_wmin-tew[j]
                tew[j]=t_wmin

                for h in range(j+1,ci) :
                    if tew[h] > t_wmin :
                        if tp > tew[h]-t_wmin :
                            tp=tp-(tew[h]-t_wmin)
                            tew[h]=t_wmin
                        else:
                            tew[h]=tew[h]-tp
                            tp=0
        water=deepcopy(tew)
        water[0]=tlw[0]

        # first ice formation
        tfreeze=(-1)*dz*L/(dzw*cwater)*first_freezeparam
        if np.mean(water[0:2]) < tfreeze and ice[0] == -np.inf :
            ice[0:3]=0
            water[0]=0

        # *******************************
        # compile results
        snow_albedo[index]=alpha_water
        ii=np.where(ice != -np.inf)
        ii=ii[0]
        ci=ii.size
        if ci > 1 :
            icethick[index]=(ci-1)*dz
            print("Icethick", icethick[index])

        if ci > 1 :
            mtemp[1,index]=np.mean(ice[ii[0]:ii[ci-2]])
        else:
            mtemp[1,index]=-np.inf
        if ci > 0 :
            stype[index]=1
        if ci > 0 :
            snow_albedo[index]=alpha_ice

        mtemp[0,index]=np.mean(water)

        l=np.where(snow[0,] != -np.inf)
        l=l[0]
        ci=l.size
        for i in range(1,ci):
            snowthick[index]=snowthick[index]+dzs/snow[1,i]
        if ci >= 1:
            snow_albedo[index]=alpha_snow
        if ci > 1 :
            mtemp[2,index]=np.mean(snow[0,l[0]:l[ci-1]])

        else:
            mtemp[2,index]=-np.inf
        if ci >= 1 :
            stype[index]=2

        hh=np.where(slush[1,] == 0)
        hh=hh[0]
        ch=hh.size
        print("Slush Counter",ch)
        if ch > 0 :
            slush_thick[index]=np.sum(slush[0,hh])*1.25
        else:
            slush_thick[index]=0
        
        print("Slush thick",slush_thick[index])
        load[0,index]=(ci-1)*dzs*1000+np.sum(slush[0,])*1000   # kg m-2
        a=7000000*pow(icethick[index],2)
        b=350000*pow(icethick[index],2)
        if load[0,index]*9.81 > b :
            load[1,index]=1
        
        if load[0,index]<0 :
            load[0,index]=0
        print('\n')
        # ********************************************
        # ********************************************
        # PLOT
        # one plot every day
        if np.mod(index,freq_plots)==0:
            plt.xlim(-40, 16)
            plt.ylim(-0.6, 0.5)
            plt.ylabel('Depth (m)')
            plt.xlabel('Temperature (deg C)')
            plt.axvline(x=0,color='k',linewidth=0.5).set_dashes([5])
            plt.axhline(y=0,color='k',linewidth=0.5)
            plt.title(str(df.loc[index,'Date'].date())+' Time '+str(df.loc[index,'When'].strftime("%H-%M")))

            if ice[0] != -np.inf:
                co=0
            else:
                co=4

            ii=np.where(ice != -np.inf)
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                yi=[float(i)*dz*(-1) for i in range(0,ci)]
                xi=ice[0:ci]
                print("Ice", te[0:ci])
                plt.plot(xi,yi,color='r',linewidth=1)
                plt.axhline(y=min(yi),color='g',linewidth=0.5)

            if ci > 0:
                offs=ii[ci-1]*dz*(-1)
            else:
                offs=0
            yw=[float(i)*dz*(-1)+offs for i in range(0,nodes[0])]
            plt.plot(water,yw,'r',linewidth=1)

            ii=np.where(snow[0,] != -np.inf)
            sc=1
            ii=ii[0]
            ci=ii.size
            if ci > 1:
                ys=[float(i)*dzs*sc for i in range(0,ci)]
                for j in range(0,ci):
                    ys[j]=ys[j]/snow[1,ii[j]]
                ys[0]=dzs/0.5
                tt=snow[0,0:ci]
                tt[0]=np.mean(tt[0:2])
                plt.axhline(y=max(ys),color='m',linewidth=0.5)
                plt.plot(tt,ys,'r',linewidth=1)

            ii=np.where(snow[0,] == -np.inf)
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                slush[0,ii]=0

            ii=np.where(np.logical_and(slush[0,] != 0,slush[1,] <= 0.1))
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                if ii[ci-1] >= len(ys):
                    n=ii[ci-1]-1
                else:
                    n=ii[ci-1]

            ii=np.where(np.logical_and(slush[0,] != 0,slush[1,] > 0.1))
            ii=ii[0]
            ci=ii.size
            if ci > 0:
                if ii[ci-1] >= len(ys):
                    n=ii[ci-1]-1
                else:
                    n=ii[ci-1]

            pdf.savefig()
            plt.close()

2017-12-06 00:00:00 Water
[4. 4. 4. 4. 4.]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 01:00:00 Water
[3.24480394 3.99907857 3.99916228 3.99916279 3.99916279]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 02:00:00 Water
[2.41666554 3.99367006 3.99831237 3.99832699 3.99832702]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 03:00:00 Water
[1.67772226 3.98341362 3.99742288 3.99749256 3.99749281]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 04:00:00 Water
[0.92231834 3.96888372 3.99646701 3.99665905 3.99666003]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 05:00:00 Water
[0.21071354 3.95004006 3.99542009 3.99582601 3.9958287 ]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 06:00:00 Water
[-0.49368959  3.92718903  3.99425841  3.99499279  3.99499874]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-12-06 07:00:00 Water
[-1.20683178  3.9004184   3.99295977  3.99415862  3.99417015]
Snow 0.0
Slush Counter 300
Slush thick 0.0


2017-1

freezing of slush at ice surface 0.0 0.0
Temp -11.036634290883631
Slush Counter2 300
Slush 0.0
JJ [-10.88980428  -5.18975263   0.        ]
[-10.88980428  -5.44490214   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-07 20:00:00 Ice 3
[-11.03663429  -5.44490214   0.                 -inf         -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -11.999263817916042
Slush Counter2 300
Slush 0.0
JJ [-11.03663429  -5.44490214   0.        ]
[-11.03663429  -5.51831715   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-07 21:00:00 Ice 3
[-11.99926382  -5.51831715   0.                 -inf         -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -13.09357410943435
Slush Counter2 300
Slush 0.0
JJ [-11.99926382  -5.51831715   0.        ]
[-11.99926382  -5.99963191   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-07 22:00:00 Ice 3
[-13.09357411  -5.99963191   0.                 -inf         -inf]
Snow 0.0
freez

freezing of slush at ice surface 0.0 0.0
Temp -13.031906785814865
Slush Counter2 300
Slush 0.0
JJ [-11.51286929  -5.2670489    0.        ]
[-11.51286929  -5.75643465   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-08 21:00:00 Ice 3
[-13.03190679  -5.75643465   0.                 -inf         -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -12.31866898539806
Slush Counter2 300
Slush 0.0
JJ [-13.03190679  -5.75643465   0.        ]
[-13.03190679  -6.51595339   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-08 22:00:00 Ice 3
[-12.31866899  -6.51595339   0.                 -inf         -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -13.412117863056563
Slush Counter2 300
Slush 0.0
JJ [-12.31866899  -6.51595339   0.        ]
[-12.31866899  -6.15933449   0.        ]
Icethick 0.01
Slush Counter 300
Slush thick 0.0


2017-12-08 23:00:00 Ice 3
[-13.41211786  -6.15933449   0.                 -inf         -inf]
Snow 0.0
freez

freezing of slush at ice surface 0.0 0.0
Temp -29.51140400299494
Slush Counter2 300
Slush 0.0
JJ [-29.46317006 -19.83785673  -9.91892836   0.        ]
[-29.46317006 -19.64211337  -9.82105669   0.        ]
Icethick 0.015
Slush Counter 300
Slush thick 0.0


2017-12-09 21:00:00 Ice 4
[-29.511404   -19.64211337  -9.82105669   0.                 -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -30.928569804306065
Slush Counter2 300
Slush 0.0
JJ [-29.511404   -19.64211337  -9.82105669   0.        ]
[-29.511404   -19.67426934  -9.83713467   0.        ]
Icethick 0.015
Slush Counter 300
Slush thick 0.0


2017-12-09 22:00:00 Ice 4
[-30.         -19.67426934  -9.83713467   0.                 -inf]
Snow 0.0
freezing of slush at ice surface 0.0 0.0
Temp -31.13375819261392
Slush Counter2 300
Slush 0.0
JJ [-30.         -19.67426934  -9.83713467   0.        ]
[-30. -20. -10.   0.]
Icethick 0.02
Slush Counter 300
Slush thick 0.0


2017-12-09 23:00:00 Ice 5
[-30. -20. -10.   0.   0.]
Snow 0.

freezing of slush at ice surface 0.0 0.0
Temp -9.492766819429281
Slush Counter2 300
Slush 0.0
JJ [-8.63503355 -8.25051695 -6.18788771 -4.12525848 -2.06262924  0.        ]
[-8.63503355 -6.90802684 -5.18102013 -3.45401342 -1.72700671  0.        ]
Icethick 0.025
Slush Counter 300
Slush thick 0.0


Ice [-8.63503355 -6.90802684 -5.18102013 -3.45401342 -1.72700671  0.        ]
2017-12-10 19:00:00 Ice 6
[-9.49276682 -6.90802684 -5.18102013 -3.45401342 -1.72700671]
Snow 0.0007000000000000001
freezing of slush at ice surface 0.0 0.0
Temp -13.304389560131115
Slush Counter2 300
Slush 0.0
JJ [-9.49276682 -6.90802684 -5.18102013 -3.45401342 -1.72700671  0.        ]
[-9.49276682 -7.59421346 -5.69566009 -3.79710673 -1.89855336  0.        ]
Icethick 0.025
Slush Counter 300
Slush thick 0.0


2017-12-10 20:00:00 Ice 6
[-13.30438956  -7.59421346  -5.69566009  -3.79710673  -1.89855336]
Snow 0.0007000000000000001
freezing of slush at ice surface 0.0 0.0
Temp -9.513674219923175
Slush Counter2 300
Slush 0.0


...

2018-01-12 13:00:00 Snow 18
[-28.15491107 -28.16224251 -28.11522993 -28.05151822 -27.97046095
 -27.85418749 -27.70115739 -27.50995118 -27.27953763 -27.00954281
 -26.70049511 -26.35402027 -25.97296363 -25.57040642 -25.14982772
 -24.71522853 -24.2710327  -23.82196169]
Snow 0.0011538111464085032
Slush Counter5 300
Slush 0.0125
Slush Counter4 300
Slush 0.0125
Slush Counter3 300
Slush 0.0125
freezing of slush at ice surface 0.0037499999999999994 0.0
Temp -27.187787578658398
More slush at 1 0.0037499999999999994
Slush Counter2 300
Slush 0.0125
JJ [-27.2242842  -27.73081796 -27.26851841 -26.80613416 -26.34362688
 -25.88096293 -25.41811471 -24.95506183 -24.49179211 -24.02830228
 -23.56459855 -23.10069684 -22.63662282 -22.17241165 -21.70810763
 -21.24376352 -20.7794398  -20.31520371 -19.85112827 -19.38729117
 -18.9237737  -18.46065956 -17.99803384 -17.53598192 -17.07458852
 -16.61393678 -16.15410748 -15.69517832 -15.2372233  -14.78031222
 -14.32451028 -13.8698777  -13.41646946 -12.96433516 -12.

2018-01-12 19:00:00 Snow 18
[-20.29033962 -20.28411744 -20.26570908 -20.24195903 -20.21366396
 -20.17834319 -20.13821964 -20.09572778 -20.05331136 -20.01321731
 -19.97730435 -19.94688449 -19.92261377 -19.90264623 -19.88660777
 -19.87381748 -19.86334264 -19.8540719 ]
Snow 0.0011538111464085032
Slush Counter5 300
Slush 0.0125
Slush Counter4 300
Slush 0.0125
Slush Counter3 300
Slush 0.0125
freezing of slush at ice surface 0.0037499999999999994 0.0
Temp -20.101793170424724
More slush at 1 0.0037499999999999994
Slush Counter2 300
Slush 0.0125
JJ [-20.10179317 -20.12377475 -19.95028287 -19.77538906 -19.59841146
 -19.41869067 -19.23559643 -19.04853374 -18.85694813 -18.66033015
 -18.45821899 -18.25020507 -18.03593179 -17.81509626 -17.58744925
 -17.35279429 -17.11098604 -16.86192806 -16.60557007 -16.34190485
 -16.07096476 -15.79281822 -15.50756601 -15.21533765 -14.91628788
 -14.61059326 -14.29844907 -13.98006636 -13.65566939 -13.32549326
 -12.98978187 -12.64878613 -12.30276243 -11.95197126 -11.

Slush Counter5 300
Slush 0.0125
Slush Counter4 300
Slush 0.0125
Slush Counter3 300
Slush 0.0125
freezing of slush at ice surface 0.0037499999999999994 0.0
Temp -20.928042677947502
More slush at 1 0.0037499999999999994
Slush Counter2 300
Slush 0.0125
JJ [-20.92804268 -20.26376879 -19.96708282 -19.67037409 -19.37362894
 -19.07683101 -18.77996052 -18.48299361 -18.18590179 -17.88865156
 -17.59120418 -17.29351556 -16.99553637 -16.69721219 -16.39848389
 -16.09928812 -15.79955783 -15.49922299 -15.1982113  -14.89644894
 -14.59386143 -14.29037437 -13.98591427 -13.68040931 -13.37379007
 -13.06599015 -12.75694684 -12.44660162 -12.1349006  -11.82179497
 -11.50724126 -11.19120161 -10.87364396 -10.55454215 -10.23387601
  -9.91163134  -9.5877999   -9.26237934  -8.93537311  -8.6067903
  -8.27664555  -7.9449588   -7.61175519  -7.27706481  -6.94092254
  -6.60336779  -6.26444436  -5.92420016  -5.58268704  -5.23996053
  -4.89607965  -4.5511067   -4.20510699  -3.85814867  -3.5103025
  -3.16164162  -2.81224

2018-01-13 07:00:00 Snow 18
[-24.86091237 -24.8714781  -24.90316532 -24.94497986 -24.99636771
 -25.06511516 -25.14959262 -25.2479313  -25.3581326  -25.47818136
 -25.60615325 -25.74030658 -25.87915005 -26.02045219 -26.16350788
 -26.30774514 -26.45270311 -26.59800064]
Snow 0.0011538111464085032
Slush Counter5 300
Slush 0.0125
Slush Counter4 300
Slush 0.0125
Slush Counter3 300
Slush 0.0125
freezing of slush at ice surface 0.0037499999999999994 0.0
Temp -25.313086006807108
More slush at 1 0.0037499999999999994
Slush Counter2 300
Slush 0.0125
JJ [-25.31393108 -24.37400233 -23.89815229 -23.42328439 -22.94987381
 -22.47837713 -22.00922686 -21.54282652 -21.07954647 -20.61972045
 -20.16364301 -19.71156776 -19.26370645 -18.82022892 -18.38126382
 -17.94689998 -17.51718848 -17.09214515 -16.67175355 -16.25596823
 -15.84471814 -15.43791022 -15.03543288 -14.63715943 -14.24295142
 -13.85266161 -13.46613679 -13.08322029 -12.70375408 -12.32758071
 -11.95454476 -11.58449413 -11.21728099 -10.85276243 -10.

2018-01-13 13:00:00 Snow 18
[-23.57408526 -23.56983044 -23.5042763  -23.41604436 -23.30476629
 -23.14781744 -22.94448863 -22.69432604 -22.39734764 -22.05425922
 -21.66664838 -21.23713512 -20.76946028 -20.27830824 -19.76764542
 -19.2419031  -18.70588733 -18.16466708]
Snow 0.0011538111464085032
Slush Counter5 300
Slush 0.0125
Slush Counter4 300
Slush 0.0125
Slush Counter3 300
Slush 0.0125
freezing of slush at ice surface 0.0037499999999999994 0.0
Temp -22.2521027153381
More slush at 1 0.0037499999999999994
Slush Counter2 300
Slush 0.0125
JJ [-22.27598539 -23.27635522 -22.93943136 -22.6019757  -22.26373413
 -21.92446634 -21.58394984 -21.24198345 -20.89839033 -20.55302032
 -20.20575163 -19.85649185 -19.50517832 -19.15177778 -18.79628549
 -18.43872379 -18.07914017 -17.717605   -17.35420894 -16.98906022
 -16.62228175 -16.25400827 -15.88438352 -15.51355763 -15.14168461
 -14.76892012 -14.39541956 -14.02133633 -13.64682057 -13.27201799
 -12.89706916 -12.52210896 -12.14726628 -11.77266395 -11.39

In [ ]:
from scipy.interpolate import interp1d

mask= df['When']==start_date_scen
scen_line=df.index[mask]

# plot time series
pp = PdfPages('./data/processed/lakeice_series.pdf')

#Ice Thickness
x=df.index
y=icethick*100
f = interp1d(x, y, kind='cubic')
x_smooth = np.linspace(x.min(), x.max(), 200)
plt.plot(x_smooth, f(x_smooth), 'red', linewidth=1)
plt.axvline(x=scen_line,color='k',linewidth=0.5).set_dashes([5])
plt.xticks(rotation='vertical')
plt.ylabel('Ice thickness(cm)')
pp.savefig()
plt.clf()

#Thickness Change Rate
r=icethick*100
rate=np.full(df.shape[0],-np.inf)
for i in range(12,df.shape[0]-12):
    r[i]=np.mean(icethick[i-12:i+13])*100

for i in range(12,df.shape[0]-12):
    rate[i]=(r[i+12]-r[i])-(r[i-12]-r[i])

rate[0:12]=(rate[12])
rate[df.shape[0]-12:df.shape[0]]=rate[df.shape[0]-13]
r=rate
for i in range(12,df.shape[0]-12):
    rate[i]=np.mean(r[i-12:i+13])

x=df.index[12:df.shape[0]-12]
y=rate[12:df.shape[0]-12]
f = interp1d(x, y, kind='cubic')
x_smooth = np.linspace(x.min(), x.max(), 200)
plt.plot(x_smooth, f(x_smooth), 'red', linewidth=1)
plt.xticks(rotation='vertical')
plt.ylabel('Ice thickness Rate(cm/day)')
#plt.plot_date(df['Date'][12:df.shape[0]-12],rate[12:df.shape[0]-12],'r',linewidth=1)
pp.savefig()
plt.clf()

#Slush Ice
x=df.index
y=slush_thick*100
f = interp1d(x, y, kind='cubic')
x_smooth = np.linspace(x.min(), x.max(), 200)
plt.plot(x_smooth, f(x_smooth), 'red', linewidth=1)
plt.xticks(rotation='vertical')
plt.ylabel('Top Ice(cm)')
pp.savefig()
plt.clf()

#Load
x=df.index
y=load[0,]
f = interp1d(x, y, kind='cubic')
x_smooth = np.linspace(x.min(), x.max(), 200)
plt.plot(x_smooth, f(x_smooth), 'red', linewidth=1)
plt.axvline(x=scen_line,color='black',linewidth=0.5).set_dashes([5])
plt.xticks(rotation='vertical')
plt.ylabel('Load(kg m-2)')
pp.savefig()
plt.clf()

# temp
x=df.index
y=df['Tair']
f = interp1d(x, y, kind='cubic')
x_smooth = np.linspace(x.min(), x.max(), 200)
plt.plot(x_smooth, f(x_smooth), 'red', linewidth=1)
plt.axvline(x=scen_line,color='black',linewidth=0.5).set_dashes([5])
plt.xticks(rotation='vertical')
plt.ylabel('Temperature (deg C)')
pp.savefig()
plt.clf()
pp.close()

In [ ]:
dfo['Icethick']=icethick*100
dfo['Tair']=df['Tair']
dfo['Rate']=rate
dfo['Slush']=slush_thick*100
dfo['Load']=load[0,]

#output csv
dfo = dfo[['Date','Time', 'Icethick', 'Rate', 'Slush', 'Load', 'Tair']]
dfo=dfo.round(1)
dfo.to_csv('./data/processed/results.csv',sep='\t')